In [1]:
from modeling.model import SimpleModel
from parsing.input_layer_extractor import InputLayerExtractor

import tensorflow as tf
import os
import numpy as np

In [2]:
model = SimpleModel(num_epochs=3)
ile = InputLayerExtractor([], '')

In [ ]:
model.model_path

In [ ]:
ile.input_layer_seed_chord.shape

In [ ]:
model.truncate_music_seed(ile.input_layer_seed_chord).shape

In [3]:
model.train()

Starting training
Epoch 0 loss: 0.693175
saving
Done


In [4]:
model.predict(ile.input_layer_seed_chord)

ValueError: Variable rnn/multi_rnn_cell/cell_0/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "modeling/model.py", line 107, in _build_graph
    initial_state=rnn_tuple_state
  File "modeling/model.py", line 177, in train
    instance_vars, model_vars = self._build_graph()
  File "<ipython-input-3-22a9f6312119>", line 1, in <module>
    model.train()


In [ ]:
ile.input_layer_seed_chord[-1]

# Predicting

In [ ]:
# Use W2, b2, states_series to make predictions
tf.reset_default_graph()

# lstm_cells = tf.get_variable("lstm_cells")
# W2 = tf.get_variable("W2", shape=[self.state_size, self.input_size])
# b2 = tf.get_variable("b2", shape=[1, self.input_size])

_init_state = np.zeros((model.num_layers, 2, 1, model.state_size))

In [ ]:
W2 = tf.get_variable("W2", shape=[10, 128])

In [ ]:
sess = tf.Session()
# Init X_seed and _current_state
init = tf.global_variables_initializer()

restore_saver = tf.train.import_meta_graph(model.model_path + '.meta')
restore_saver.restore(sess, tf.train.latest_checkpoint(os.path.dirname(model.model_path)))

In [ ]:
tf.reset_default_graph()

W2 = tf.get_variable("W2", shape=[10, 128])
_init_state = np.zeros((model.num_layers, 2, 1, model.state_size))

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(
        sess,
        model.model_path
    )
#         os.path.join(
#             os.path.dirname(model.model_path),
#             'checkpoint'
#         )
#     )
    print(W2.eval())

In [ ]:
W2.eval(session=sess)

In [ ]:
X_seed = tf.placeholder(
    tf.float32,
    [1, model.truncated_backprop_length * model.input_size],
    name='X'
)
_current_state = tf.placeholder(
    tf.float32,
    [model.num_layers, 2, 1, model.state_size]
)

In [ ]:
state_per_layer_list = tf.unstack(_current_state, axis=0)

In [ ]:
rnn_tuple_state = tuple(
    [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0,:,:], state_per_layer_list[idx][1,:,:])
    for idx in range(model.num_layers)]
)

In [ ]:
stacked_rnn = []
for _ in range(model.num_layers):
    stacked_rnn.append(tf.nn.rnn_cell.LSTMCell(
        model.state_size, state_is_tuple=True
    ))
lstm_cells = tf.nn.rnn_cell.MultiRNNCell(
    stacked_rnn,
    state_is_tuple=True
)

In [ ]:
state_per_layer_list

In [ ]:
stacked_rnn

In [ ]:
states_series, current_state = tf.nn.dynamic_rnn(
    lstm_cells,
    tf.expand_dims(X_seed, -1),
    initial_state=rnn_tuple_state
)

In [ ]:
states_series

In [ ]:
128 * 25

In [ ]:
sess.run('W2')

In [ ]:
logits = tf.matmul(states_series, W2) + b2

logits = tf.reshape(
    logits,
    [1, self.input_size * self.truncated_backprop_length, self.input_size],
)



In [ ]:
logits_series = tf.unstack(logits, axis=1)
predictions_series = [
    tf.nn.sigmoid(logits) for logits in logits_series
]

In [ ]:
# Restore model variables
sess.run(init)

In [ ]:

next_pred = self._single_pred()
result = sess.run(next_pred, feed_dict={
    X_seed: X_seed_np,
    _current_state: _init_state
})

In [ ]:
for i in range(0, 6, 2):
    print i